# Telco Classificatin Project - Working

### Project Goals

The goal of this project is to identify drivers of customer churn and and make a recommend changes that will increase customer retention.


### Project Description

Customer churn is a problem that is costing Telco revenue. This project attempts to identify reasons for customer churn in order to increase customer retention and reduce loss of revenue. The knowledge gained could expand beyond customer retention to improve sales by identifying what customers like and dislike. This project will anaylize the attributes of customers that churn to build a model that will predict the probablity of a customer churning given specific attributes. 

### Initial Questions

How many customers are churning and not churning?

What is the timeline of churn?

At what month of tenure are they churning?

What non-monetary drivers are associated with churned customers?

What monetary drivers are associated with churned customers?

Do monetary drivers outweigh non-monetary drivers?

What drivers are easiest to change with least amount of impact to the customer? 

### Get Data
- Added env.py to the repo .gitignore file so env.py won't be pushed to github
- Created an env file that contains my username, password, and host address fo the MySQL Server.

In [1]:
# import pandas and numpy to start coding
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# import env file for hostname, username, password, and db_name
from env import host, user, password, db_name

In [ ]:
# Pass env file authentication to container 'url'
url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [ ]:
# define sql search for all records from all tables
sql = '''
    SELECT * FROM customers c
    JOIN contract_types ct ON ct.contract_type_id=c.contract_type_id
    JOIN internet_service_types ist ON ist.internet_service_type_id=c.internet_service_type_id
    JOIN payment_types pt ON pt.payment_type_id=c.payment_type_id
    '''

In [ ]:
# pull sql records using url and sql containers
df = pd.read_sql(sql,url)
df.head()

In [ ]:
# create function 'get_connection' for repeated use to pass authentication to MySQL server
def get_connection(db_name):
    '''
   This function used the passed database name and imports host, user, password
   from the locally saved env file to authenticate with the MySQL server.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [ ]:
# create function 'get_telco' for repeated use.
def get_telco():
    '''
    This function uses the the get_connection function to pull the telco data from the MySQL server. 
    '''
    sql = '''
    SELECT * FROM customers c
    JOIN contract_types ct ON ct.contract_type_id=c.contract_type_id
    JOIN internet_service_types ist ON ist.internet_service_type_id=c.internet_service_type_id
    JOIN payment_types pt ON pt.payment_type_id=c.payment_type_id
    '''
    url = get_connection('telco_churn')
    df = pd.read_sql(sql, url)
    return df

In [2]:
# test functions that was placed in acquire_telco.py: get_connection, get_telco
import acquire_telco
df = acquire_telco.get_telco()
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,internet_service_type_id,internet_service_type,payment_type_id,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,1,DSL,4,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,1,DSL,3,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,1,DSL,4,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,1,DSL,3,Bank transfer (automatic)


##  <font color = 'red'> Clean Data

In [ ]:
# look for object columns that need to be removed or converted to dummy values or differenty data types
df.info()

#### Found duplicated columns:
- payment_type_id
- internet_service_type_id
- contract_type_id

In [ ]:
df[df.duplicated()] # no duplicated rows

In [ ]:
# dropping ducplicate columns
df = df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'])
df.info()

In [ ]:
# check total charges column
df.total_charges.head()

#### Errored when trying to convert total_charges to float. Found blank vlaues in column.

In [ ]:
# Check values of total_charges, found 21 row had blanks
df[df.total_charges == ' ']

In [ ]:
# Replaced found blanks found with 0
df['total_charges'] = df.total_charges.replace(' ', '0')

In [ ]:
# Check values of total_charges are not blank
df[df.total_charges == ' ']

In [ ]:
# Convert total_charges to float
df.total_charges = df.total_charges.astype(float)
df.info()

In [ ]:
 # id columns for dummy variables
dummies_cols =[
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'phone_service',
    'multiple_lines',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'paperless_billing',
    'churn',
    'contract_type',
    'internet_service_type',
    'payment_type']

In [ ]:
# create dummy variables and assign to variable
dummies_df = pd.get_dummies(df[dummies_cols], drop_first=True)

In [ ]:
# concat df with dummy variable columns
df = pd.concat([df, dummies_df], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
# Create Function to Clean Data
def clean_telco(df):
    
    # dropping ducplicate columns
    df = df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'])
    
    #replace blanks in total_charges with 0
    df['total_charges'] = df.total_charges.replace(' ', '0')
    
    # Convert total_charges to float
    df.total_charges = df.total_charges.astype(float)
    
     # id columns for dummy variables
    dummies_cols =[
        'gender',
        'partner',
        'dependents',
        'phone_service',
        'multiple_lines',
        'online_security',
        'online_backup',
        'device_protection',
        'tech_support',
        'streaming_tv',
        'streaming_movies',
        'paperless_billing',
        'churn',
        'contract_type',
        'internet_service_type',
        'payment_type']
    
    # Create dummy variables for id'd columns
    dummies_df = pd.get_dummies(df[dummies_cols], drop_first=True)
    
    # Concat dummy vairabls to df
    df = pd.concat([df, dummies_df], axis=1)
    
    # Clean column names
    df.columns = [col.lower().replace(' ','_') for col in df]
    
    return df

In [3]:
import prep_telco

# test function clean_telco
df = prep_telco.clean_telco(df)
df.head()

/Users/kanieladenis/codeup-data-science/telco_classification_project/prep_telco.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_charges'] = df.total_charges.replace(' ', '0')
/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/kanieladenis/codeup-data-science/telco_classification_project/prep_telco.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,churn_no,churn_yes,contract_type_month_to_month,contract_type_one_year,contract_type_two_year,internet_service_type_dsl,internet_service_type_fiber_optic,internet_service_type_none,payment_auto,payment_manual
0,0016-QLJIS,Female,no,Yes,Yes,65,Yes,Yes,Yes,Yes,...,1,0,0,0,1,1,0,0,0,1
1,0017-DINOC,Male,no,No,No,54,No,No phone service,Yes,No,...,1,0,0,0,1,1,0,0,1,0
2,0019-GFNTW,Female,no,No,No,56,No,No phone service,Yes,Yes,...,1,0,0,0,1,1,0,0,1,0
3,0056-EPFBG,Male,no,Yes,Yes,20,No,No phone service,Yes,No,...,1,0,0,0,1,1,0,0,1,0
4,0078-XZMHT,Male,no,Yes,No,72,Yes,Yes,No,Yes,...,1,0,0,0,1,1,0,0,1,0


#### clean_telco function tested good and was added to the prep_telco file

## <font color='red'>Data Context

### How many customers in each subgroups

In [ ]:
cols =  ['gender_female',
 'gender_male',
 'partner_no',
 'partner_yes',
 'dependents_no',
 'dependents_yes',
 'phone_service_no',
 'phone_service_yes',
 'multiple_lines_no',
 'multiple_lines_no_phone_service',
 'multiple_lines_yes',
 'online_security_no',
 'online_security_no_internet_service',
 'online_security_yes',
 'online_backup_no',
 'online_backup_no_internet_service',
 'online_backup_yes',
 'device_protection_no',
 'device_protection_no_internet_service',
 'device_protection_yes',
 'tech_support_no',
 'tech_support_no_internet_service',
 'tech_support_yes',
 'streaming_tv_no',
 'streaming_tv_no_internet_service',
 'streaming_tv_yes',
 'streaming_movies_no',
 'streaming_movies_no_internet_service',
 'streaming_movies_yes',
 'paperless_billing_no',
 'paperless_billing_yes',
 'churn_no',
 'churn_yes',
 'contract_type_month_to_month',
 'contract_type_one_year',
 'contract_type_two_year',
 'internet_service_type_dsl',
 'internet_service_type_fiber_optic',
 'internet_service_type_none',
 'payment_type_bank_transfer_(automatic)',
 'payment_type_credit_card_(automatic)',
 'payment_type_electronic_check',
 'payment_type_mailed_check']

In [ ]:
for col in cols:
    print(col)
    print(df[col].sum())

### How many Customers are Churning? 
- Total Customer: 7043, Churned: 1,869, Not-Churned: 5,174

In [ ]:
# Total Customer: 7043, Churned: 1,869, Not-Churned: 5,174
df.churn.value_counts()

In [ ]:
# 1/3 of retained customers churn
df.churn.hist()

### What is the timeline of churn? At what month of tenure are they churning?
- Tenure consist of 72 months. Most Churn Happens in the first 5 months. Almost 3 times as much churn in month 1 as in month 2.

In [ ]:
# Tenure ranges from 0 month to 72 months
df.tenure.max()

In [ ]:
df_churn_tenure = df.groupby(by='tenure').churn_yes.sum().reset_index()
df_churn_tenure.head()

In [ ]:
# Show Timeline for Churn
plt.figure(figsize=(20,10))
sns.barplot(data=df_churn_tenure, x='tenure', y='churn_yes')
plt.title('Most Churn Hapens in the first 5 months')
plt.xlabel('Tenure Month')
plt.ylabel('Customers Churned')
plt.show()

## <font color = 'red'> Split and Prep Data

In [ ]:
# import split module 
from sklearn.model_selection import train_test_split

# Split data to create test set
train_validate, test = train_test_split(df,
                                        test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)

In [ ]:
# Split data to create validate and train sets
train, validate = train_test_split(train_validate,
                                      test_size=.3,
                                      random_state=123,
                                      stratify=train_validate.churn)

In [ ]:
# confirm split
train.shape, validate.shape, test.shape

In [ ]:
# Create function to split data
def split_telco(df):
    '''
    This function splits the telco data into the train, validate, and test samples at 
    portions: train= 56%, validate= 24%, test = 20%
    '''
    
    # Split data to create test sample
    train_validate, test = train_test_split(df, 
                                             test_size=.2, 
                                             random_state=123, 
                                             stratify=df.churn)
    
    # Split data to create train and validate samples
    train, validate = train_test_split(train_validate,
                                      test_size=.3,
                                      random_state=123,
                                      stratify=train_validate.churn)
    
    return train, validate, test

In [5]:
# Test split_telco function and add to prep_telco file
train, validate, test = split_telco(df)
train.shape, validate.shape, test.shape

((3943, 65), (1691, 65), (1409, 65))

## <font color = 'red'> Split Data

In [ ]:
# test imported function
import prep_telco
train, validate, test = prep_telco.split_telco(df)
train.shape, validate.shape, test.shape

## <font color ='red'> Explore Train Data to answer questions
- What attributes are associated with churned customers?
- Do monetary drivers outweigh non-monetary drivers?
- What drivers are easiest to change with least amount of impact to the customer? 

In [ ]:
# id columns to keep
cols_counts = [
    'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'tenure',
 'phone_service',
 'multiple_lines',
    'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'monthly_charges',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type',
]

In [ ]:
# create subset of train df
train_cols = train[cols_counts]

In [ ]:
# create histograms for each column id'd
for col in train_cols:
    plt.hist(train_cols[col])
    plt.title(col)
    plt.show()

Histograms notes for significant subgroups:
- Gender are almost even
- Seniors are half as many as non-seniors
- No significant difference between partners to non-partners
- Over twice as many non-dependents to dependents
- majority of tenure is at 0 month and 70 month, no significant difference in the between  months
- 6 times as many customer have phone service than don't
- No sig difference between muliple line
- Twice as many do not have online_security as do. No internet service is half of no online security
- No sig diff between online_backup
- No sig diff between device_protection
- Less than twice as many do not have tech_support than do
- No sig diff between streaming tv
- No sig diff between steaming movies
- Pareless billing are a third more than paperbilling
- Most customers have a monthly charge less than 30 dollars
- Month-to-Month contracts more than double the one-year and two-year contracts 
- Fiber Optic as about 25% more customers. Half as many customers have no internet service
- Auto Payemtents have 1/3 more customers than others

### From the above notes, below are variables that have significan difference
- Over twice as many non-dependents to dependents
- 6 times as many customer have phone service than don't
- Twice as many do not have online_security as do. No internet service is half of no online security
- Less than twice as many do not have tech_support than do
- Pareless billing are a third more than paperbilling
- Most customers have a monthly charge less than 30 dollars
- Month-to-Month contracts more than double the one-year and two-year contracts 
- Fiber Optic as about 25% more customers. Half as many customers have no internet service
- Auto Payemtents have 1/3 more customers than others

### Subgroups to check against Churn
- dependent_yes = 0
- phone_service_yes = 1
- online_security = 0
- tech_support_yes = 0
- paperless_billing = 1
- monthly_charge < 30
- contract_type = 0
- internet_service_type
- payment_type
- tenure

### <font color = 'red'> What attributes are associated with churned customers with <= 5 months of tenure?

In [ ]:
train.churn_yes.sum()

In [ ]:
train.payment_type.value_counts()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='dependents')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='phone_service')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='online_security')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='tech_support')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='paperless_billing')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='total_charges')

In [ ]:
train_monthly30 = train[train.monthly_charges <= 30]
plt.figure(figsize=(15,5))
sns.barplot(data=train_monthly30, y='churn_yes', x='monthly_charges')

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=train, y='churn_yes', x='monthly_charges')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='contract_type')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='internet_service_type')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='payment_type')

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=train, y='churn_yes', x='tenure')

### Attibutes associated with sigificant churn:
- Yes Month-to-Month Contract_Type
- Yes Fiber Optic Internet_Service_Type
- No Tech Support
- No Online Security
- No Dependents
- Yes E-checking
- Yes Paperless Billing=
### Focus on:
- Month-to-Month Churn = 928 customers

In [ ]:
train.contract_type.value_counts()

In [ ]:
train[train['contract_type_month-to-month'] == 1].churn_yes.sum()

In [ ]:
train[train['contract_type_one_year'] == 1].churn_yes.sum()

In [ ]:
train[train['contract_type_two_year'] == 1].churn_yes.sum()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data=train, x='contract_type', y='churn_yes')
plt.title('Three Times as Many Customers have Monthly Contracts than Annual Contracts')
plt.xlabel('Contract Types')
plt.ylabel('Churn Count')
plt.show()

### There are 2,162 month-to-month contracts, more than double of the next highest (two year contract). 928 of which are churned customers.

In [ ]:
# Create subgroup for all month-to-month contracts
train_m2m = train[train.contract_type == 'Month-to-month']
train_m2m.head()

### Multivariate Analysis from the above attributes

In [ ]:
# Find churn sum or online_security_no
(train_m2m.online_security_no == 1).sum()

In [ ]:
# Find churn sum for internet_service_type_fiber_optic
(train_m2m.internet_service_type_fiber_optic == 1).sum()

In [ ]:
# Find churn sum for tech_support_no
(train_m2m.tech_support_no == 1).sum()

In [ ]:
# Find churn sum for dependents_no
(train_m2m.dependents_no == 1).sum()

In [ ]:
# Find churn sum for payment_type_electronic_check
(train_m2m.payment_type_electronic_check == 1).sum()

### Find churn counts for above contracts

In [ ]:
# Find churn sum or online_security_no
m2m[m2m.online_security_no == 1].churn_yes.sum()

In [ ]:
# Find churn sum for internet_service_type_fiber_optic
mm2m[m2m.internet_service_type_fiber_optic == 1].churn_yes.sum()

In [ ]:
# Find churn sum for tech_support_no
m2m[m2m.tech_support_no == 1].churn_yes.sum()

In [ ]:
# Find churn sum for dependents_no
m2m[m2m.dependents_no == 1].churn_yes.sum()

In [ ]:
# Find churn sum for payment_type_electronic_check
echeck = m2m[m2m.payment_type_electronic_check == 1].churn_yes.sum()
echeck

### Investigate difference for making payment_type autopay vs manual pay

In [ ]:
mcheck = m2m[m2m.payment_type_mailed_check == 1].churn_yes.sum()
mcheck

In [ ]:
echeck + mcheck

### Discovered 'Manual Pay' and 'No Dependents' has more Churn than the other attributes

In [ ]:
# Update train, validate, test with new columns for manual pay and auto pay
train['auto_pay'] = (train['payment_type_bank_transfer_(automatic)'] == 1) | (train['payment_type_credit_card_(automatic)'] == 1)
train['manual_pay'] = (train['payment_type_electronic_check'] == 1) | (train['payment_type_mailed_check'] == 1)

validate['auto_pay'] = (validate['payment_type_bank_transfer_(automatic)'] == 1) | (validate['payment_type_credit_card_(automatic)'] == 1)
validate['manual_pay'] = (validate['payment_type_electronic_check'] == 1) | (validate['payment_type_mailed_check'] == 1)

test['auto_pay'] = (test['payment_type_bank_transfer_(automatic)'] == 1) | (test['payment_type_credit_card_(automatic)'] == 1)
test['manual_pay'] = (test['payment_type_electronic_check'] == 1) | (test['payment_type_mailed_check'] == 1)

In [ ]:
# Created barpolot to compare churn bettween customers using Manual Pay and Not Using Manual Pay
plt.figure(figsize=(15,5))
sns.barplot(data=train, y='churn_yes', x='manual_pay')
plt.title('Churn for Manual Payment Customer double Churn from Auto Payment Customers')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.show()

In [ ]:
# 762 Churned under manual pay
train[train.manual_pay == 1].churn_yes.sum()

In [ ]:
# 284 Churned under Auto Pay
train[train.auto_pay == 1].churn_yes.sum()

In [ ]:
# Shows higher CHURN for month-to-month contracts using manual_pay and NO tech_support
x = 'manual_pay'
y = 'churn_yes'
hue = 'tech_support_no'
plt.figure(figsize=(15,5))
sns.barplot(data=train_m2m, hue=hue, x=x, y=y)
plt.title('Highest Churn from Manual Payment Customers with NO Tech Support')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.show()

In [ ]:
# Shows higher CHURN for month-to-month contracts using Manual Pay and NO Online Security
x = 'manual_pay'
y = 'churn_yes'
hue = 'online_security_no'
plt.figure(figsize=(15,5))
sns.barplot(data=train_m2m, hue=hue, x=x, y=y)
plt.title('Highest Churn from Manual Payment Customers with NO Online Security')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.show()

In [ ]:
# Shows significantly higher CHURN for month-to-month contracts using Manual Pay and Fiber Optic
x = 'manual_pay'
y = 'churn_yes'
hue = 'internet_service_type_fiber_optic'
plt.figure(figsize=(15,5))
sns.barplot(data=train_m2m, hue=hue, x=x, y=y)
plt.title('Churn is highest with Manual Payment Customers that have Fiber Optic Internet')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.show()

In [ ]:
# Shows higher CHURN for month-to-month contract using Manual Pay and has NO dependents
x = 'manual_pay'
y = 'churn_yes'
hue = 'dependents_no'
plt.figure(figsize=(15,5))
sns.barplot(data=train_m2m, hue=hue, x=x, y=y)
plt.title('Churn is highest for Manual Payments Customers with NO Dependents')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.show()

## Questions Answered:
How many customers are churning and not churning?
- Total Customer: 7043, Churned: 1,869, Not-Churned: 5,174

What is the timeline of churn?
- Tenure ranges  to 72 months

At what month of tenure are they churning?
- Most Churn within 5 months

What are major subgroups of customers?
- Month-to-Month Contracts

What people drivers are associated with churned customers?
- Not having dependents

What product  drivers are associated with churned customers?
- Having Fiber Optic Internet
- Not having Tech support
- Not having Online Security
- Using E-Checking (Manual Pay)

Using E-checking is misleading, investigate splitting payment methods into auto pay vs manual pay
- Manual Pay Churn is Double of Auto Pay Churn

What attributes associate to the highest Churn?
- Having No Dependents
- Using Manual Pay

Do people drivers outweigh Product drivers?
- Single factoring, no. Individually, people attributes have more churn however there are more product attributes associated to churn.

What drivers are easiest to change with least amount of impact to the customer? 
- Change Manual Pay to Auto Pay focused on customers with no Dependents

### Provide 5-min Visuals for exploration (two with statistical test)

## One Sample T-Test if Avg Manual Pay Churn is Greater than the Avg Overall Churn for M2M contracts

In [ ]:
# Variance is not equal
df_mpay = train_m2m[train_m2m.manual_pay == 1]
print(df_mpay.churn_yes.var())
print(train_m2m.churn_yes.var())

In [ ]:
# normal distribution: Yes, more than 30 observations
# Independent Samples: Yes
# Var Equal: No
# H0: Avg Churn of Manual pay is less than equal to Overall Avg Churn
# HA: Avg Churn of Manual Pay is greater than Overall Avg Churn
avg_overall_churn = train_m2m.churn_yes.mean()
manual_pay_churn = df_mpay.churn_yes
alpha = .05

t, p = stats.ttest_1samp(manual_pay_churn, avg_overall_churn)
t, p/2, alpha

### Reject H0, it seems that avg churn for manual pay is greater than overall average churn for M2M contracts

In [ ]:
# Shows higher CHURN for month-to-month contract using Manual Pay and has NO dependents
x = 'manual_pay'
y = 'churn_yes'
plt.figure(figsize=(15,5))
sns.barplot(data=train_m2m, x=x, y=y)
plt.title('Average Churn for Manual Payment Customers are Greater than Overall Average of Churn')
plt.xlabel('Manual Payment Customers')
plt.ylabel('Churn Count')
plt.axhline(y=avg_overall_churn, color='red')
plt.show()

## Independent T-Test if Avg Churn of Manual Pay with No Dependents is Greater than Avg Churn of Manual Pay with No Online Security for M2M Contracts

### Independent T-Test if Avg Manual Pay with No Dependents Churn is Greater than Avg Manual Pay with No Online Security
- Normal distribution: Yes, more than 30 observations
- Independent Samples: Yes
- Var Equal: No
- H0: Avg Churn of Manual Pay with No Dependents <= Avg Churn of Manual Pay with No Online Security
- HA: Avg Churn of Manual Pay with No Dependents > Avg Churn of Manual Pay with No Online Security

In [ ]:
mp_nd = df_mpay[(df_mpay.dependents_no == 1)]
mp_nd.head()

In [ ]:
mp_ndc = mp_nd.churn_yes
mp_ndc

In [ ]:
mp_nos = df_mpay[df_mpay.online_security_no == 1]
mp_nosc = mp_nos.churn_yes
mp_nosc

In [ ]:
print(mp_ndc.var())
print(mp_nosc.var())

In [ ]:
alpha = .05
t, p = stats.ttest_ind(mp_ndc, mp_nosc, equal_var=False)
t, p/2, alpha

### Since t < 0, Fail Reject of H0. It seems that Avg Churn for online_security_no is <= Avge Churn for dependents_no

In [ ]:
df_mpay.online_security_no.head()

In [ ]:
# Shows Churn for online_security_no > Churn for dependents_no
y = 'churn_yes'
x = 'dependents_no'
hue = 'online_security_no'
plt.figure(figsize=(15,5))
sns.barplot(data=df_mpay, y=y, x=x, hue=hue )
plt.title('Customers Without Online Security Churn More than Customers with No Dependents ')
plt.xlabel('Customers with No Dependents')
plt.ylabel('Churn Count')
plt.show()

### Summarize Exploration

### During exploration, I discoverd that 27% of our customers churn, and most of that happens whithin the first five months. I learned that tthe largest subgroup of our customers belong to monthly contracts. I learned that significant drivers associated with customers that churn include:
- Having No Dependents
- Having Fiber Optic Internet
- Not having Tech support
- Not having Online Security
- Using Manual Pay
### Of those attributes, customers that use Manual Pay have the highest Churn followed by Customers without dependents and customers without online security.
### Further exploration revealed that people drivers like customer that have not dependents and customers that use manual payments have more churn than product drivers. Note however that they are more product driver.
### If monthly contracts using manual payments are subgrouped, then customers without online security churn more tha customers without dependents
### Given the explored subgroups and attributes associated with churn, the recommendation going forward will also consider what is the easiest to change with the least impact to customer. I leaning to toward promoting auto pay to customer without dependents.
### Features NOT being used for Modeling:
** Most Features Did Not Make a Significant Subgroup ** 
- Customer ID: Not needed
- Gender: No significant subgroup difference between male and female
- Partner: No significant subgroup difference between partner status subgroup
- Multiple Phone Lines: No sigfnificant subgroup difference between categories
- Online Backup: No significant subgroup difference between categories
- Device Protection: No significant subgroup difference between categories
- Streaming TV: No significant subgroup differnce between categories
- Streaming Movies: No significant subgroup difference between categories
- Phone Service: No significant churn differnce between categories
- Monthly Charges: Unable to differentiate between values
- Total Charges: Unable to differentiate between values

### Features USED for modeling:
** All Features Show Significant Difference in Churn Between Categories **
- Dependents: Favored No
- Tenure: Most Churn occurs within five months
- Tech Support: Favored no
- Online Security: Favored No
- Contract Type: Baselined to Month-to-Month
- Payment Type: Converted to Auto Pay and Manual Pay
- Internet Service Type:Favored Fiber Optic

## Modeling

### Drop Unusable and Unneeded Columns for Modelingb

In [ ]:
drop_cols = [
    'customer_id',
 'gender',
'partner',
 'dependents',
'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
'churn',
 'contract_type',
 'internet_service_type',
 'payment_type',
 'gender_female',
 'gender_male',
'phone_service_no',
 'phone_service_yes',
 'multiple_lines_no',
 'multiple_lines_no_phone_service',
 'multiple_lines_yes',
 'online_backup_no',
 'online_backup_no_internet_service',
 'online_backup_yes',
 'device_protection_no',
 'device_protection_no_internet_service',
 'device_protection_yes',
'streaming_tv_no',
 'streaming_tv_no_internet_service',
 'streaming_tv_yes',
 'streaming_movies_no',
 'streaming_movies_no_internet_service',
 'streaming_movies_yes',
'monthly_charges',
 'total_charges',
'partner_yes',
 'dependents_yes',
'online_security_no_internet_service',
'tech_support_no_internet_service',
'paperless_billing_no',
'churn_no',
'contract_type_two_year',
'internet_service_type_none',
'payment_type_bank_transfer_(automatic)',
 'payment_type_credit_card_(automatic)',
 'payment_type_electronic_check',
 'payment_type_mailed_check',
 'auto_pay']

In [ ]:
train = train.drop(columns=drop_cols)
train.columns.to_list()

In [ ]:
validate = validate.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)

In [6]:
# run prep_teco to remove columns for modeling
train, validate, test = prep_telco.prep_telco(train,validate,test)

### Create y_train and X_train sets

In [ ]:
X_train = train.drop(columns=['churn_yes'])
y_train = train.churn_yes

X_validate = validate.drop(columns=['churn_yes'])
y_validate = validate.churn_yes

X_test = test.drop(columns=['churn_yes'])
y_test = test.churn_yes

In [7]:
# run functino to create x,y versions for train, validate, test
X_train, y_train, X_validate, y_validate, X_test, y_test = prep_telco.xy_version(train,validate,test)

### Select Evaluation Metric (Report.ipynb)

#### Churn is positive class
#### Not Churn is negative class
- TP: Predict Churn and the customer is Churn
- TN: Predict Not Churn and the customer is Not Churn
- FP: Predict Churn and the customer is Not Churn
- FN: Predict Not Churn and the customer is Churn
#### Emphasize Actuals (Recall) so that if prediction is wrong Telco still has a customer which will increase reatainability

### Evaluate Baseline (Report.ipynb)

In [ ]:
# Find Most Frequent Churn Value for Baseline =
train.churn_yes.mode() #Most Frequent Value is Zero

In [ ]:
# Create Baseline by setting all values to zero
baseline = (y_train == 0).mean()
baseline

### Develop 3 Models (Report.ipynb)

#### Creat Model 1 - Decision Tree 

In [8]:
# Using Decision Tree because it is easier to understand
# Create Decision Tree Classifier Algorithym Container
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

# Fit Train Data to Container to make the model
clf = clf.fit(X_train, y_train)

# Create Prediction Series
y_pred_clf = clf.predict(X_train)

#### Create Model 2 - Random Forest

In [9]:
# Using Random Forest to see if Accuracy and Recall increase
# Create Random Forest Classifier Algorithym Container
rf = RandomForestClassifier(max_depth=3, random_state=123)

# Fit Train Data to Container to make the model
rf = rf.fit(X_train, y_train)

# Create Prediction Series
y_pred_rf = rf.predict(X_train)

#### Create Model 3 - K-Nearest Neighbor

In [10]:
# Using KNN to see if Accuracy and Recall increase
# Create Decision Tree Classifier Algorithym Container
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

# Fit Train Data to Container to make the model
knn = knn.fit(X_train, y_train)

# Create Prediction Series
y_pred_knn = knn.predict(X_train)

### Evaluate on Train (Report.ipynb)

#### Evaluate Model 1 - Decision Tree

In [ ]:
#Confusion Matrix for Model 1
labels = sorted(y_train.unique())
print('Actual on Left, Predicted on Top')
pd.DataFrame(confusion_matrix(y_train, y_pred1), index=labels, columns=labels)

Confusion Matrix Breakdown
- TP: 386
- TN: 2737
- FP: 160
- FN: 660

In [ ]:
# Create Classification Report for Model 1
print(classification_report(y_train, y_pred1))

In [17]:
# Test for range of max_depth, found 5 gets recall of .5 and accuracy of .8 without overfit
for i in range(1,20):
    clf_i = DecisionTreeClassifier(max_depth=i, random_state=123)
    clf_i = clf_i.fit(X_train, y_train)
    y_pred_clf_i = clf_i.predict(X_train)
    score = clf_i.score(X_train, y_train)
    report = classification_report(y_train, y_pred_clf_i)
    print(f'----max_depth = {i} ------------------------')
    print(f' Accuracy Score: {score} ')
    print(f' Report : {report} ')

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


----max_depth = 1 ------------------------
 Accuracy Score: 0.7347197565305605 
 Report :               precision    recall  f1-score   support

           0       0.73      1.00      0.85      2897
           1       0.00      0.00      0.00      1046

    accuracy                           0.73      3943
   macro avg       0.37      0.50      0.42      3943
weighted avg       0.54      0.73      0.62      3943
 
----max_depth = 2 ------------------------
 Accuracy Score: 0.7605883844788233 
 Report :               precision    recall  f1-score   support

           0       0.86      0.81      0.83      2897
           1       0.54      0.62      0.58      1046

    accuracy                           0.76      3943
   macro avg       0.70      0.72      0.71      3943
weighted avg       0.77      0.76      0.77      3943
 
----max_depth = 3 ------------------------
 Accuracy Score: 0.792036520415927 
 Report :               precision    recall  f1-score   support

           0       0

#### Accuracy better than baseline by .06. Recall is at .37


#### Evaluate Model 2 - Random Forest

In [ ]:
##Confusion Matrix for Model 2
labels = sorted(y_train.unique())
print('Actual on Left, Predicted on Top')
pd.DataFrame(confusion_matrix(y_train, y_pred2), index=labels, columns=labels)

Confusion Matrix Breakdown
- TP: 336
- TN: 2772
- FP: 125
- FN: 710

In [ ]:
# Create Classification Report for Model 2
print(classification_report(y_train, y_pred2))

#### Accuracy is the same as Decision Tree but Recall has droped by .05


#### Evaluate Model 3 - K-Nearest Neighbor

In [ ]:
##Confusion Matrix for Model 3
labels = sorted(y_train.unique())
print('Actual on Left, Predicted on Top')
pd.DataFrame(confusion_matrix(y_train, y_pred3), index=labels, columns=labels)

Confusion Matrix Breakdown
- TP: 622
- TN: 2657
- FP: 240
- FN: 424

In [ ]:
# Create Classification Report for Model 2
print(classification_report(y_train, y_pred3))

#### Accuracy increased by .04 to .83 and Recall increased .17 to .53

In [19]:
# Test for range of max_depth, found 5 gets recall of .5 and accuracy of .8 without overfit
for i in range(1,20):
    rf_i = RandomForestClassifier(max_depth=i, random_state=123)
    rf_i = rf_i.fit(X_train, y_train)
    y_pred_rf_i = rf_i.predict(X_train)
    report = classification_report(y_train, y_pred_rf_i)
    print(f'----max_depth = {i} ------------------------')
    print(report)

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


----max_depth = 1 ------------------------
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      2897
           1       0.00      0.00      0.00      1046

    accuracy                           0.73      3943
   macro avg       0.37      0.50      0.42      3943
weighted avg       0.54      0.73      0.62      3943

----max_depth = 2 ------------------------
              precision    recall  f1-score   support

           0       0.77      0.98      0.86      2897
           1       0.79      0.19      0.31      1046

    accuracy                           0.77      3943
   macro avg       0.78      0.59      0.59      3943
weighted avg       0.78      0.77      0.72      3943

----max_depth = 3 ------------------------
              precision    recall  f1-score   support

           0       0.80      0.96      0.87      2897
           1       0.74      0.32      0.45      1046

    accuracy                           0.79      3943

In [21]:
# Tested Range for neighbors. Shows that 5 neibhors is around best for accuracy and recall
for i in range(1,20):
    knn = KNeighborsClassifier(n_neighbors=i, weights='uniform')
    knn = knn.fit(X_train, y_train)
    y_pred = knn.predict(X_train)
    report = classification_report(y_train, y_pred)
    print(f'n_neighbor = {i}')
    print(report)

n_neighbor = 1
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      2897
           1       0.82      0.85      0.84      1046

    accuracy                           0.91      3943
   macro avg       0.88      0.89      0.89      3943
weighted avg       0.91      0.91      0.91      3943

n_neighbor = 2
              precision    recall  f1-score   support

           0       0.82      0.99      0.90      2897
           1       0.93      0.42      0.58      1046

    accuracy                           0.84      3943
   macro avg       0.88      0.70      0.74      3943
weighted avg       0.85      0.84      0.81      3943

n_neighbor = 3
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      2897
           1       0.75      0.66      0.70      1046

    accuracy                           0.85      3943
   macro avg       0.82      0.79      0.80      3943
weighted avg       0.85      

### KNN Model works best with default hyperparameters (5 neighbors) resulting in .59 recall and .83 accuracy


### Evaluate on Validate (Report.ipynb)

#### Evaluate Validate on Random Forest with max_depth=5

In [ ]:
rf5 = RandomForestClassifier(max_depth=5, random_state=123)
rf5 = rf.fit(X_train, y_train)
y_pred = rf5.predict(X_validate)
report = classification_report(y_validate, y_pred)
print(report)

In [ ]:
baseline

#### Random Forest with Max Depth 5 results in .79 Accuracy and .53 Recall, several point lower than train inidicating it is not overfit


#### Evaluate Validate on K-Nearest Neighbor with n_neighbor set to 5

In [20]:
knn5 = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn5 = knn5.fit(X_train, y_train)
y_pred = knn5.predict(X_validate)
report = classification_report(y_validate, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1242
           1       0.59      0.53      0.56       449

    accuracy                           0.78      1691
   macro avg       0.71      0.70      0.70      1691
weighted avg       0.77      0.78      0.77      1691



#### KNN Model with 5 neighbors using validate set results in .77 accuracy and .51 recall, serveral point lower than train indicating it is not overfit


### Evaluate Top Model on Test (Report.ipynb)
#### Random Forest beat KNN on both accuracy and recall by .02 each. Going forward with Test

In [ ]:
# Using Random Forest with max_depth=5 on Test Sample
y_pred = rf5.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

#### Random Forest with max_depth=5 on Test Sample resulted in .80 Accuracy and .58 Recall, near the results of validate

## Report (Final Notebook) 
- code commenting (Report.ipynb)
- markdown (Report.ipynb)
- no errors when running

### Written Conclusion Summary (Report.ipynb)
Churn is definetly a problem. Out of the 7,043 customers, 1,869 have churned, a rate of 27%. 
After filtering down subgroups to month-to-month contract, and then focusing on customers using manual pay, I am targeting 2,743 customers of which 1,276 have churned, a 47% churn rate. 

### Conclusion Recommendations (Report.ipynb)
Focus promoting customers on the month to month contract, using manual payment, to convert to auto payment. Auto Pay has 1,132 customers with month-to-month contract using auto pay of which 379 churn, a 33% churn rate. This recommendation has the potential to reduce churn for that subgroup by 14%, retaining 370 more customers.

### Conclusion Next steps (Report.ipynb)
With more time, I would explore more into how to retain customers past 1 month of tenure. There are 613 customers with 1 month tenure, 380 of which churn, a 62% churn rate.

In [ ]:
df.columns

In [ ]:
m2m = df[df['contract_type_month-to-month'] == 1]
m2m_mp = m2m[(m2m['payment_type_electronic_check'] == 1) | (m2m['payment_type_mailed_check'] == 1)]
m2m_mp.churn_yes.sum()

In [ ]:
m2m = df[df['contract_type_month-to-month'] == 1]
m2m_ap = m2m[(m2m['payment_type_credit_card_(automatic)'] == 1) | (m2m['payment_type_bank_transfer_(automatic)'] == 1)]
m2m_ap.churn_yes.sum()

In [ ]:
m2m_ap.shape

## Live Presentation

- intro (live)
- audience & setting  (live)
- content (live)
- Verbal Conclusion (findings, next steps, recommendations)  (live) 
- time (live) 

## Deliver Predictions
- Deliver predictions (.csv) 